## Importação das bibliotecas

In [61]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
import os

## Criação do Dataframe com Spark

In [ ]:
spark = ( 
 SparkSession
 .builder
 .master("local[*]")
 .appName('spark_dataframe_api')
 .getOrCreate()
)

data_path = "caminho/dos/arquivos/csv"

file_paths = [os.path.join(data_path, file) for file in os.listdir(data_path) if file.endswith('.csv')]

# Inicializa um DataFrame vazio
final_df = None

# Lê e combina os arquivos usando union
for file in file_paths:
    print(f"Arquivo encontrado: {os.path.basename(file)}")

    temp_df = spark.read.option("delimiter", ";").option("header", "true").option("inferSchema", "true").csv(file)
    
    if final_df is None:
        final_df = temp_df
    else:
        final_df = final_df.union(temp_df)

# Verifica o schema final
final_df.printSchema()

df = final_df


Arquivo encontrado: ca-2020-01.csv
Arquivo encontrado: ca-2020-02.csv
Arquivo encontrado: ca-2021-01.csv
Arquivo encontrado: ca-2021-02.csv
Arquivo encontrado: ca-2022-02.csv
Arquivo encontrado: precos-semestrais-ca-2022-01.csv
Arquivo encontrado: Preços semestrais - AUTOMOTIVOS_2023.01.csv
Arquivo encontrado: Preços semestrais - AUTOMOTIVOS_2023.02.csv
Arquivo encontrado: Preços semestrais - AUTOMOTIVOS_2024.01.csv
Arquivo encontrado: Preços semestrais - AUTOMOTIVOS_2024.02.csv
root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- V

In [6]:
df.show(5, truncate=False)

+--------------+--------------+---------+----------------------------+-------------------+------------------------+----------+-------------------+----------+---------+----------+--------------+--------------+---------------+-----------------+----------------------------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda                     |CNPJ da Revenda    |Nome da Rua             |Numero Rua|Complemento        |Bairro    |Cep      |Produto   |Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira                    |
+--------------+--------------+---------+----------------------------+-------------------+------------------------+----------+-------------------+----------+---------+----------+--------------+--------------+---------------+-----------------+----------------------------+
|SE            |SP            |GUARULHOS|AUTO POSTO SAKAMOTO LTDA    | 49.051.667/0001-02|RODOVIA PRESIDENTE DUTRA|S/N       |KM 210,5-SENT SP/RJ|BONSUCESSO|07178-580|GASOLINA  |03/01/

## Filtrar Colunas

In [34]:
df_filtered = (
    df.select('Estado - Sigla', 'Municipio', 'Bairro', 'Revenda', 'Bandeira', 'Produto', 'Valor de Compra', 'Valor de Venda', 'Unidade de Medida')
)

df_filtered.show(5)

+--------------+---------+----------+--------------------+--------------------+----------+---------------+--------------+-----------------+
|Estado - Sigla|Municipio|    Bairro|             Revenda|            Bandeira|   Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+---------+----------+--------------------+--------------------+----------+---------------+--------------+-----------------+
|            SP|GUARULHOS|BONSUCESSO|AUTO POSTO SAKAMO...|PETROBRAS DISTRIB...|  GASOLINA|           NULL|         4,399|       R$ / litro|
|            SP|GUARULHOS|BONSUCESSO|AUTO POSTO SAKAMO...|PETROBRAS DISTRIB...|    ETANOL|           NULL|         3,199|       R$ / litro|
|            SP|GUARULHOS|BONSUCESSO|AUTO POSTO SAKAMO...|PETROBRAS DISTRIB...|DIESEL S10|           NULL|         3,899|       R$ / litro|
|            SP|GUARULHOS|BONSUCESSO|AUTO POSTO SAKAMO...|PETROBRAS DISTRIB...|       GNV|           NULL|         2,995|          R$ / m³|
|            BA| SAL

## Configuração inicial

In [62]:
# Ajusta a coluna "Valor de Venda" em todo o DataFrame
df = df.withColumn("Valor de Venda", F.regexp_replace("Valor de Venda", ",", ".").cast(DoubleType()))

## Perguntas

### Cálculo da Média, Mediana e Desvio Padrão dos preços de venda da Gasolina, Etanol, Diesel e Diesel S10.

In [63]:
result = (
    df.groupBy("Produto")
      .agg(
          F.avg("Valor de Venda").alias("Média"),                   # Média
          F.percentile_approx("Valor de Venda", 0.5).alias("Mediana"),  # Mediana
          F.stddev("Valor de Venda").alias("Desvio Padrão")         # Desvio padrão
      )
)

result.show()

+------------------+-----------------+-------+------------------+
|           Produto|            Média|Mediana|     Desvio Padrão|
+------------------+-----------------+-------+------------------+
|        DIESEL S10|5.381306979685092|   5.68|1.2708373344671733|
|            DIESEL|5.292243032869912|   5.59|1.2642657559466581|
|            ETANOL|4.156815503138976|  3.999|0.8830197250592021|
|               GNV|4.390210353724093|   4.49|0.8303960347825217|
|          GASOLINA|5.600917728050836|   5.65|0.9349063082810379|
|GASOLINA ADITIVADA|6.049327963982642|   5.99|0.7489803901236937|
+------------------+-----------------+-------+------------------+



### Quais são os 3 principais postos de São Paulo que têm a maior média de venda da Gasolina, Etanol e Diesel

In [64]:
df_sp = df.filter(F.col("Estado - Sigla") == 'SP')

result = (
    df_sp.groupBy("Produto", "Revenda") 
        .agg(F.avg("Valor de Venda").alias("media_venda"))
        .orderBy("Produto", F.desc("media_venda"))
)

# Seleciona os 3 principais postos para cada produto
top_3_postos = (
    result.groupBy("Produto")
        .agg(F.collect_list("Revenda").alias("top_postos"))
        .withColumn("top_postos", F.expr("slice(top_postos, 1, 3)"))  # Pega os 3 primeiros de cada produto
)

# Exibe os resultados
top_3_postos.show(truncate=False)

+------------------+--------------------------------------------------------------------------------------------------------------------------+
|Produto           |top_postos                                                                                                                |
+------------------+--------------------------------------------------------------------------------------------------------------------------+
|DIESEL S10        |[PEDRO BRANCO FERRARO LTDA, AUTO POSTO FERRARO LTDA, MARIO VICENTE SERVICOS AUTOMOTIVOS LTDA]                             |
|DIESEL            |[POSTO GAIVOTA LTDA, AUTO POSTO NEMO LTDA, AUTO POSTO SANTA CATARINA DE OURINHOS - EIRELI]                                |
|GASOLINA ADITIVADA|[AUTO POSTO LEAO DA ALDEIA LTDA, SENA PARK AUTO POSTO LTDA, SERVICOS AUTOMOTIVOS EMBUENSE LTDA]                           |
|ETANOL            |[CLEAN CAR SUPER LAVAGEM AUTOM�TICA E COMERCIO LTDA, POSTO DE SERVI�OS DOMINANTE LTDA, AUTO POSTO BRASIL DE CA�APAVA

### Qual o estado que possui a maior média de venda para Diesel e Diesel S10

In [65]:
df_filtered_diesel = df.filter(df["Produto"] == 'DIESEL')
df_filtered_diesel_s10 = df.filter(df["Produto"] == 'DIESEL S10')

result_diesel = (
    df_filtered_diesel.groupBy("Estado - Sigla")
    .agg(F.avg("Valor de Venda").alias("media_venda"))
    .orderBy(F.desc("media_venda"))
)

result_diesel_s10 = (
    df_filtered_diesel_s10.groupBy("Estado - Sigla")
    .agg(F.avg("Valor de Venda").alias("media_venda"))
    .orderBy(F.desc("media_venda"))
)

print("Diesel")
result_diesel.show(1)  # Exibe o estado no topo da lista

print("Diesel S10")
result_diesel_s10.show(1)

Diesel
+--------------+-----------------+
|Estado - Sigla|      media_venda|
+--------------+-----------------+
|            AC|6.133045477344927|
+--------------+-----------------+
only showing top 1 row

Diesel S10
+--------------+-----------------+
|Estado - Sigla|      media_venda|
+--------------+-----------------+
|            AC|6.173175639617018|
+--------------+-----------------+
only showing top 1 row



### Quais foram os valores de venda mais alto atrelados a cada bandeira do estado de São Paulo

In [67]:
df_sp = df.filter(df["Estado - Sigla"] == 'SP')

result = (
    df_sp.groupBy("Bandeira")
         .agg(F.max("Valor de Venda").alias("valor_maximo"))  # Calcula o valor máximo de venda
         .orderBy("Bandeira")
)

result.show(truncate=False)

+----------------------------+------------+
|Bandeira                    |valor_maximo|
+----------------------------+------------+
|ALE                         |7.69        |
|ALESAT                      |8.04        |
|BRANCA                      |8.49        |
|CIAPETRO                    |7.69        |
|D`MAIS                      |6.49        |
|GRAN PETRO                  |4.339       |
|IPIRANGA                    |9.65        |
|LIQUIGÁS                    |6.44        |
|PETROBRAS DISTRIBUIDORA S.A.|6.899       |
|PETROBRASIL                 |6.39        |
|PODIUM                      |8.499       |
|PR DISTRIBUIDORA            |4.499       |
|RAIZEN                      |8.99        |
|RIO BRANCO                  |7.69        |
|RM PETROLEO                 |4.079       |
|RODOIL                      |6.39        |
|ROYAL FIC                   |4.259       |
|RUFF C.J.                   |7.53        |
|SMALL                       |6.99        |
|TAURUS                      |7.

### Qual o município apresentou o maior e o menor preço médio do Diesel?

In [ ]:
df_filled = df.fillna({"Valor de Venda": 0})
df_diesel = df_filled.filter(df["Produto"] == "DIESEL")

result = (
    df_diesel.groupBy("Municipio")
             .agg(F.avg("Valor de Venda").alias("media_venda"))
             .orderBy("media_venda")
)

print("Menor preço médio de venda do Diesel")
result.orderBy(F.asc("media_venda")).show(1, truncate=False)

print("Maior preço médio de venda do Diesel")
result.orderBy(F.desc("media_venda")).show(1, truncate=False) 

Menor preço médio de venda do Diesel
+--------------+-----------------+
|Municipio     |media_venda      |
+--------------+-----------------+
|PESCARIA BRAVA|3.262323529411765|
+--------------+-----------------+
only showing top 1 row

Maior preço médio de venda do Diesel
+---------+-----------------+
|Municipio|media_venda      |
+---------+-----------------+
|TEFE     |7.164734104046243|
+---------+-----------------+
only showing top 1 row



### Informe os 3 bairros de Recife que apresentaram a maior média de preço para Diesel e Diesel S10, e seus respectivos preços

In [71]:
df_recife_diesel = df.filter(
    (F.trim(df["Produto"]).isin("DIESEL", "DIESEL S10")) & (F.trim(df["Municipio"]) == "RECIFE")
)

df_recife_diesel = df_recife_diesel.withColumn(
    "Valor de Venda", F.regexp_replace("Valor de Venda", ",", ".").cast("double")
)

result = (
    df_recife_diesel.groupBy("Bairro")
    .agg(F.avg("Valor de Venda").alias("media_venda"))
    .orderBy(F.desc("media_venda"))
)

result_clean = result.filter(result["media_venda"].isNotNull())

top_3_bairros = result_clean.limit(3)
top_3_bairros.show(truncate=False)

+---------+-----------------+
|Bairro   |media_venda      |
+---------+-----------------+
|PAISSANDU|7.19             |
|HIPODROMO|5.577326315789474|
|SOLEDADE |5.533424242424242|
+---------+-----------------+



## Analise sobre o "Valor de Venda"

In [72]:
df_price = (
    df_filtered
    .groupBy(
        F.col('Estado - Sigla'),
        F.col('Produto'),
        F.col('Unidade de Medida')
    )
    .agg(
        F.min(F.col("Valor de Venda")).alias('menor_valor'),
        F.max(F.col("Valor de Venda")).alias('maior_valor')
    )
    .withColumn(
        "diferenca",
        F.col('maior_valor') - F.col('menor_valor')
    )
    .orderBy('diferenca', ascending=False)
)

df_price.show(10)

+--------------+------------------+-----------------+-----------+-----------+---------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|diferenca|
+--------------+------------------+-----------------+-----------+-----------+---------+
|            TO|               GNV|          R$ / m³|          3|          3|      0.0|
|            AC|            DIESEL|       R$ / litro|       3,34|       8,95|     NULL|
|            AC|        DIESEL S10|       R$ / litro|       3,55|       8,98|     NULL|
|            AC|            ETANOL|       R$ / litro|       3,45|        7,9|     NULL|
|            AC|          GASOLINA|       R$ / litro|       3,99|       8,16|     NULL|
|            AC|GASOLINA ADITIVADA|       R$ / litro|       4,92|       9,69|     NULL|
|            AL|            DIESEL|       R$ / litro|      2,799|       8,99|     NULL|
|            AL|        DIESEL S10|       R$ / litro|      2,799|       8,99|     NULL|
|            AL|            ETAN